# Clean - Clean Dataset
This example illustrate how to perform meta-blocking on a clean-clean dataset (data linkage), so when we have two datasets that do not contain duplicates and we want to discover the duplicates between them. 

In [1]:
import sparker
import random
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

2022-03-31 23:29:26,985 WARN util.Utils: Your hostname, aditi-OMEN-by-HP-Laptop-16-b0xxx resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
2022-03-31 23:29:26,985 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-31 23:29:27,291 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-31 23:29:27,769 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-03-31 23:29:27,769 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Load the data
sparkER provides wrappers to load CSV and JSON files.

First, we load the first dataset, and we extract the maximum id. The profiles ids of the second dataset will be assigned starting from this one.

*real_id_field* is the field that contains the identifier of the record.

In [3]:
# Profiles contained in the first dataset
profiles1 = sparker.CSVWrapper.load_profiles(file_path = '/Final_Project/sample/Sample1_cleaned.csv',
                                             start_id_from = 0,
                                             separator=",", header=True,
                                             real_id_field = "id",
                                            source_id = 0)
# Max profile id in the first dataset, used to separate the profiles in the next phases
separator_id = profiles1.map(lambda profile: profile.profile_id).max()
# Separators, used during blocking to understand from which dataset a profile belongs. It is an array because sparkER
# could work with multiple datasets
separator_ids = [separator_id]

In [4]:
type(separator_id)

int

In [5]:
separator_id

29

In [6]:
profiles1.take(5)
profiles1.toDF().show(1364)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

In [7]:
a = profiles1.take(29)
a

[{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'DisplayName', 'value': 'Geoff Dalgas'}, {'key': 'Location', 'value': 'Corvallis, OR'}, {'key': 'AboutMe', 'value': 'Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing. Find me

Let's visualize a profile to check if they are correctly loaded

In [8]:
print(profiles1.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 0}


Loads the second dataset and extract the max id (it will be used later)

In [9]:
profiles2 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample2_cleaned.csv', 
                                              start_id_from = separator_id+1, 
                                              real_id_field = "id",
                                            source_id = 1)
# Max profile id
max_profile_id = profiles2.map(lambda profile: profile.profile_id).max()

In [10]:
profiles2.count()

30

In [11]:
profiles2.toDF().show(3226)

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, ta...|          0|        30|        1|
|[{DisplayName, Al...|          1|        31|        1|
|[{DisplayName, ja...|          2|        32|        1|
|[{DisplayName, fw...|          3|        33|        1|
|[{DisplayName, Ma...|          4|        34|        1|
|[{DisplayName, Be...|          5|        35|        1|
|[{DisplayName, Ji...|          6|        36|        1|
|[{DisplayName, sa...|          7|        37|        1|
|[{DisplayName, ja...|          8|        38|        1|
|[{DisplayName, Da...|          9|        39|        1|
|[{DisplayName, Pa...|         10|        40|        1|
|[{DisplayName, va...|         11|        41|        1|
|[{DisplayName, Ph...|         12|        42|        1|
|[{DisplayName, Av...|         13|        43|        1|
|[{DisplayName, He...|         14|        44|   

In [12]:
separator_id_1 = profiles2.map(lambda profile: profile.profile_id).max()
separator_ids = [separator_id, separator_id_1]
separator_ids

[29, 59]

In [15]:
profiles3 = sparker.CSVWrapper.load_profiles('/Final_Project/sample/Sample2_cleaned.csv',
                                              start_id_from = separator_id_1+1,
                                              real_id_field = "id",
                                            source_id = 2)

Finally, concatenate the two RDDs

In [16]:
#profiles = profiles1.union(profiles2)

In [17]:
profile_1 = profiles1.union(profiles2)
profiles = profile_1.union(profiles3)

In [18]:
profiles.take(2)

[{'profile_id': 0, 'attributes': [{'key': 'DisplayName', 'value': 'Community'}, {'key': 'Location', 'value': 'on the server farm'}, {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."}, {'key': 'Age', 'value': '27'}, {'key': 'phone', 'value': '655-452-4816'}], 'original_id': '0', 'source_id': 0},
 {'profile_id': 1, 'attributes': [{'key': 'DisplayName', 'value': 'Geoff Dalgas'}, {'key': 'Location', 'value': 'Corvallis, OR'}, {'key': 'AboutMe', 'value': 'Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing. Find me

In [14]:
profiles.count()

60

In [19]:
profiles.toDF().show(4590) #b000jz4hqo

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

In [20]:
newColumns = ["attributes","original_id","profile_id","source_id"]
p = spark.createDataFrame(data = profiles, schema = newColumns)

In [21]:
p.show()

+--------------------+-----------+----------+---------+
|          attributes|original_id|profile_id|source_id|
+--------------------+-----------+----------+---------+
|[{DisplayName, Co...|          0|         0|        0|
|[{DisplayName, Ge...|          1|         1|        0|
|[{DisplayName, Ji...|          2|         2|        0|
|[{DisplayName, St...|          3|         3|        0|
|[{DisplayName, tx...|          4|         4|        0|
|[{DisplayName, Ke...|          5|         5|        0|
|[{DisplayName, Em...|          6|         6|        0|
|[{DisplayName, To...|          7|         7|        0|
|[{DisplayName, Se...|          8|         8|        0|
|[{DisplayName, Fr...|          9|         9|        0|
|[{DisplayName, Br...|         10|        10|        0|
|[{DisplayName, cm...|         11|        11|        0|
|[{DisplayName, kā...|         12|        12|        0|
|[{DisplayName, Sc...|         13|        13|        0|
|[{DisplayName, Ma...|         14|        14|   

In [22]:
tokens_per_profile = profiles.map(lambda profile: (
            profile.attributes))

In [23]:
tokens_per_profile.take(2)

[[{'key': 'DisplayName', 'value': 'Community'},
  {'key': 'Location', 'value': 'on the server farm'},
  {'key': 'AboutMe', 'value': "Hi, I'm not really a person. I'm a background process that helps keep this site clean! I do things like Randomly poke old unanswered questions every hour so they get some attention. Own community questions and answers so nobody gets unnecessary reputation from them. Own downvotes on spam/evil posts that get permanently deleted. Own suggested edits from anonymous users. Remove abandoned questions."},
  {'key': 'Age', 'value': '27'},
  {'key': 'phone', 'value': '655-452-4816'}],
 [{'key': 'DisplayName', 'value': 'Geoff Dalgas'},
  {'key': 'Location', 'value': 'Corvallis, OR'},
  {'key': 'AboutMe', 'value': 'Dev #2 who helped create Stack Overflow currently working at Microsoft. I love all people willing to share their knowledge. Without community we are nothing. Find me on: Twitter. Stack Overflow ValuedAssociate #00003'},
  {'key': 'Age', 'value': '24'},
 

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [24]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('/Final_Project/sample/ground_truth_sample.csv',
                                        'id1', 'id2')

In [25]:
gt.toDF().show(1400)

+---------------+----------------+
|first_entity_id|second_entity_id|
+---------------+----------------+
|             20|               0|
|             21|               1|
|             22|               2|
|             23|               3|
|             24|               4|
|             25|               5|
|             26|               6|
|             27|               7|
|             28|               8|
|             29|               9|
+---------------+----------------+



In [26]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles1, profiles2)

In [27]:
new_gt

{(20, 30),
 (21, 31),
 (22, 32),
 (23, 33),
 (24, 34),
 (25, 35),
 (26, 36),
 (27, 37),
 (28, 38),
 (29, 39)}

In [28]:
len(new_gt)

10

In [29]:
# We can explore some pairs
random.sample(new_gt, 5)

/tmp/ipykernel_21044/3641384983.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(new_gt, 5)


[(23, 33), (27, 37), (28, 38), (21, 31), (25, 35)]

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [30]:
# blocks = sparker.Blocking.create_blocks(profiles, separator_ids, 
#                                         blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
#                                         ngram_size=4)
# print("Number of blocks",blocks.count())

In [31]:
# sparker.BlockingKeysStrategies.ngrams_blocking

In [32]:
# blocks.take(500)

Let's continue by using the standard token blocking

In [33]:
blocks = sparker.Blocking.create_blocks(profiles, separator_ids)
print("Number of blocks",blocks.count())

Number of blocks 554


In [35]:
blocks.take(500)

[{'block_id': 0, 'profiles': [{0, 3, 8, 10, 12, 15, 17, 23, 24, 25, 28, 29}, {33, 34, 35, 38, 39, 40, 41, 43, 46, 57, 58}, {64, 65, 68, 69, 70, 71, 73, 76, 87, 88, 63}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 1, 'profiles': [{0, 1, 24}, {41, 34}, {64, 71}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0, 8, 24}, {34}, {64}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0}, {43}, {73}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0, 9}, {41, 47}, {77, 71}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 17, 14}, {55}, {85}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{1}, {59}, {89}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 7, 'profiles': [{1, 25}, {35}, {65}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 

In [36]:
blocks.take(1)

[{'block_id': 0, 'profiles': [{0, 3, 8, 10, 12, 15, 17, 23, 24, 25, 28, 29}, {33, 34, 35, 38, 39, 40, 41, 43, 46, 57, 58}, {64, 65, 68, 69, 70, 71, 73, 76, 87, 88, 63}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''}]

## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [40]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

2022-04-01 00:26:16,395 ERROR executor.Executor: Exception in task 0.0 in stage 45.0 (TID 47)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aditi/DBDA_HOME/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/aditi/DBDA_HOME/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 417, in func
    return f(iterator)
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 2146, in combineLocally
    

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 47) (192.168.0.115 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/aditi/DBDA_HOME/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/aditi/DBDA_HOME/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/aditi/anaconda3/lib/python3.9/site-packages/pyspark/rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/adi

In [33]:
blocks_purged.count()

288

In [34]:
blocks_purged.take(500)

[{'block_id': 1, 'profiles': [{0, 3}, {43}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0, 10}, {57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 24}, {41, 34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0}, {41}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 8, 3, 14}, {43}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0}, {52}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 7, 'profiles': [{0}, {59}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 8, 'profiles': [{0, 8, 24}, {34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 9, 'profiles': [{0, 8, 14, 24}, {34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 10, 'profiles': [{0, 9}, {41, 47}], 'entropy': -1.0, 'clus

In [39]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8, separator_ids)

NameError: name 'blocks_purged' is not defined

In [36]:
profile_blocks_filtered.count()

60

In [37]:
profile_blocks_filtered.take(500)

[{profile_id: 0, blocks: {{'block_id': 5, 'comparisons': 4}, {'block_id': 6, 'comparisons': 1}, {'block_id': 2, 'comparisons': 2}, {'block_id': 147, 'comparisons': 4}, {'block_id': 8, 'comparisons': 3}, {'block_id': 149, 'comparisons': 3}, {'block_id': 9, 'comparisons': 4}, {'block_id': 144, 'comparisons': 1}, {'block_id': 10, 'comparisons': 4}, {'block_id': 4, 'comparisons': 1}, {'block_id': 146, 'comparisons': 2}, {'block_id': 1, 'comparisons': 2}, {'block_id': 7, 'comparisons': 1}, {'block_id': 11, 'comparisons': 1}}},
 {profile_id: 10, blocks: {{'block_id': 53, 'comparisons': 6}, {'block_id': 153, 'comparisons': 6}, {'block_id': 157, 'comparisons': 2}, {'block_id': 54, 'comparisons': 1}, {'block_id': 51, 'comparisons': 2}, {'block_id': 154, 'comparisons': 2}, {'block_id': 46, 'comparisons': 2}, {'block_id': 49, 'comparisons': 1}, {'block_id': 158, 'comparisons': 7}, {'block_id': 45, 'comparisons': 4}, {'block_id': 2, 'comparisons': 2}, {'block_id': 47, 'comparisons': 2}, {'block_id

In [38]:
blocks_purged.take(500)

[{'block_id': 1, 'profiles': [{0, 3}, {43}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 2, 'profiles': [{0, 10}, {57}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 3, 'profiles': [{0, 1, 24}, {41, 34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 4, 'profiles': [{0}, {41}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 5, 'profiles': [{0, 8, 3, 14}, {43}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 6, 'profiles': [{0}, {52}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 7, 'profiles': [{0}, {59}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 8, 'profiles': [{0, 8, 24}, {34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 9, 'profiles': [{0, 8, 14, 24}, {34}], 'entropy': -1.0, 'cluster_id': -1, 'blocking_key': ''},
 {'block_id': 10, 'profiles': [{0, 9}, {41, 47}], 'entropy': -1.0, 'clus

In [39]:
print("Number of blocks_purged",blocks_purged.count())

Number of blocks_purged 288


If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [38]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt, separator_ids)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

NameError: name 'blocks_after_filtering' is not defined

## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [79]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [80]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [81]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 1.0
Precision 0.10416666666666667
Number of comparisons 96


In [82]:
num_matches

10

The number of edges
The number of matches (only if the groundtruth is provided)
The retained edges

In [83]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show()

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          6|            0|[{0, 52, 1}, {0, ...|
|         10|            0|[{10, 57, 3}, {10...|
|          3|            1|[{24, 34, 56}, {2...|
|          0|            0|                  []|
|          8|            0|[{8, 41, 2}, {8, ...|
|          3|            0|[{14, 55, 2}, {14...|
|          0|            0|                  []|
|          0|            0|                  []|
|          1|            0|        [{2, 35, 2}]|
|          0|            0|                  []|
|          3|            1|[{28, 58, 3}, {28...|
|          0|            0|                  []|
|          1|            1|      [{26, 36, 16}]|
|          0|            0|                  []|
|          5|            0|[{12, 39, 1}, {12...|
|          0|            0|                  []|
|          0|            0|                  []|
|          1|       

### Reciprocal Weighted Node Pruning

In [47]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 43


In [48]:
newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          3|            0|[{0, 43, 3}, {0, ...|
|          5|            0|[{10, 43, 2}, {10...|
|          1|            0|      [{24, 34, 56}]|
|          0|            0|                  []|
|          5|            0|[{8, 55, 2}, {8, ...|
|          2|            0|[{14, 55, 2}, {14...|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|            0|                  []|
+-----------+-------------+--------------------+
only showing top 10 rows



### Weighted Edge Pruning

In [49]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          separator_ids,
                          weight_type=sparker.WeightTypes.CBS,
                          #groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
# print("Recall", num_matches/len(new_gt))
# print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Number of comparisons 21


In [50]:



newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(10)

+-----------+-------------+--------------------+
|no_of_edges|no_of_matches|       retaind_edges|
+-----------+-------------+--------------------+
|          1|            0|        [{0, 43, 3}]|
|          3|            0|[{10, 57, 3}, {10...|
|          1|            0|      [{24, 34, 56}]|
|          0|            0|                  []|
|          1|            0|        [{8, 34, 4}]|
|          1|            0|       [{14, 34, 4}]|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|            0|                  []|
|          0|            0|                  []|
+-----------+-------------+--------------------+
only showing top 10 rows



In [51]:
lst1newColumns = ["no_of_edges","no_of_matches","retaind_edges"]
results.toDF(newColumns).show(truncate=False)

+-----------+-------------+---------------------------------------+
|no_of_edges|no_of_matches|retaind_edges                          |
+-----------+-------------+---------------------------------------+
|1          |0            |[{0, 43, 3}]                           |
|3          |0            |[{10, 57, 3}, {10, 48, 3}, {10, 35, 3}]|
|1          |0            |[{24, 34, 56}]                         |
|0          |0            |[]                                     |
|1          |0            |[{8, 34, 4}]                           |
|1          |0            |[{14, 34, 4}]                          |
|0          |0            |[]                                     |
|0          |0            |[]                                     |
|0          |0            |[]                                     |
|0          |0            |[]                                     |
|2          |0            |[{28, 38, 15}, {28, 58, 3}]            |
|0          |0            |[]                   

## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [84]:
edges = results.flatMap(lambda x: x[2])

edges.count()

96

In [85]:
newColumns = ["profile_id","neighbor_id","neighbor_weight"]
ans = edges.toDF(newColumns)
# ans.sort_values(by=['neighbor_weight'])    #show(50000)

In [86]:

ans = spark.createDataFrame(data = edges, schema = newColumns)

In [87]:
ans.show(50000)

+----------+-----------+---------------+
|profile_id|neighbor_id|neighbor_weight|
+----------+-----------+---------------+
|         0|         42|              1|
|         0|         43|              3|
|         0|         41|              2|
|         0|         34|              2|
|         0|         59|              2|
|         0|         52|              1|
|        10|         35|              3|
|        10|         41|              2|
|        10|         56|              2|
|        10|         57|              3|
|        10|         48|              3|
|        10|         40|              1|
|        10|         50|              1|
|        10|         42|              1|
|        10|         49|              1|
|        10|         43|              2|
|        24|         34|             56|
|        24|         42|              1|
|        24|         40|              1|
|         8|         34|              4|
|         8|         47|              2|
|         8|    

In [88]:
# ans.groupBy("profile_id").max("neighbor_weight").show()

In [89]:
"""reduceByKey
# Merge the values for each key using an associative and commutative reduce function.

# This will also perform the merging locally on each mapper before sending results to a reducer, similarly to a “combiner” in MapReduce
"""


'reduceByKey\n# Merge the values for each key using an associative and commutative reduce function.\n\n# This will also perform the merging locally on each mapper before sending results to a reducer, similarly to a “combiner” in MapReduce\n'

In [90]:
max_by_group = (edges
  .map(lambda x: (x[0], x))  
   .reduceByKey(lambda x1, x2: max(x1, x2, key=lambda x: x[-1]))
   .values())
                
# Convmax_by_group.collect()ert to PairwiseRD
  # Take maximum of the passed arguments by the last element (key)
  # equivalent to:
  # lambda x, y: x if x[-1] > y[-1] else y

In [91]:
max_by_group.take(60)

[(0, 43, 3),
 (10, 48, 3),
 (24, 34, 56),
 (8, 34, 4),
 (14, 34, 4),
 (2, 35, 2),
 (28, 38, 15),
 (26, 36, 16),
 (12, 39, 1),
 (20, 30, 11),
 (22, 32, 9),
 (18, 34, 1),
 (16, 40, 1),
 (4, 47, 1),
 (6, 41, 1),
 (3, 43, 3),
 (1, 41, 2),
 (9, 34, 1),
 (25, 35, 45),
 (17, 34, 4),
 (7, 53, 2),
 (11, 56, 2),
 (19, 59, 2),
 (21, 31, 8),
 (29, 39, 28),
 (5, 57, 2),
 (15, 34, 3),
 (13, 31, 1),
 (27, 37, 6),
 (23, 33, 12)]

In [92]:
edges.take(10)

[(0, 43, 3),
 (0, 34, 2),
 (0, 59, 2),
 (0, 42, 1),
 (0, 41, 2),
 (0, 52, 1),
 (10, 35, 3),
 (10, 57, 3),
 (10, 43, 2),
 (10, 56, 2)]

In [93]:
# lst1 = []
# dataCollect = ans.collect()
# for row in dataCollect:
#     lst1.append(row[0])
    
# list_set = set(lst1)
# s = list(list_set)
# s

In [94]:
# lst0 = []
# p_id = []
# n_id = []
# e = []
# d = {}

# def cal():
#     for i in s:
#         maxm = 1
#         for row in dataCollect:
#             if(i == row[0]):
#                 if(row[2] > maxm):
#                     maxm = row[2]
#                     #print(f"{row[0]}, {row[1]}, {row[2]}")
#                     p_id.append(row[0])
#                     n_id.append(row[1]-(separator_id+1))
#                     e.append(row[2])
#                     d = {"P_id": p_id, "N_id": n_id, "weight": e}
#     return(d)

# maxm = 0
# dataCollect = ans.collect()

# a = cal()
# df =  pd.DataFrame.from_dict(a)
# df

#             #lst1.append(row[0])

In [95]:
dataCollect = max_by_group.collect()
df =  pd.DataFrame(dataCollect)
df

0   1   2
0    0  43   3
1   10  48   3
2   24  34  56
3    8  34   4
4   14  34   4
5    2  35   2
6   28  38  15
7   26  36  16
8   12  39   1
9   20  30  11
10  22  32   9
11  18  34   1
12  16  40   1
13   4  47   1
14   6  41   1
15   3  43   3
16   1  41   2
17   9  34   1
18  25  35  45
19  17  34   4
20   7  53   2
21  11  56   2
22  19  59   2
23  21  31   8
24  29  39  28
25   5  57   2
26  15  34   3
27  13  31   1
28  27  37   6
29  23  33  12

In [96]:
df.rename(columns = {0:'P_id', 1:'N_id',
                              2:'weight'}, inplace = True)
df.head()

P_id  N_id  weight
0     0    43       3
1    10    48       3
2    24    34      56
3     8    34       4
4    14    34       4

In [97]:
n = df.filter(["P_id","N_id","weight"])
n = pd.DataFrame(n)
n

P_id  N_id  weight
0      0    43       3
1     10    48       3
2     24    34      56
3      8    34       4
4     14    34       4
5      2    35       2
6     28    38      15
7     26    36      16
8     12    39       1
9     20    30      11
10    22    32       9
11    18    34       1
12    16    40       1
13     4    47       1
14     6    41       1
15     3    43       3
16     1    41       2
17     9    34       1
18    25    35      45
19    17    34       4
20     7    53       2
21    11    56       2
22    19    59       2
23    21    31       8
24    29    39      28
25     5    57       2
26    15    34       3
27    13    31       1
28    27    37       6
29    23    33      12

In [98]:
n["N_id"] = n["N_id"]-(separator_id+1)
new_df = n[(n['weight']>7)]
new_df

P_id  N_id  weight
2     24     4      56
6     28     8      15
7     26     6      16
9     20     0      11
10    22     2       9
18    25     5      45
23    21     1       8
24    29     9      28
29    23     3      12

In [99]:
col = ["P_id","N_id","weight"]
df_weight = spark.createDataFrame(new_df, schema=col)
df_weight.show()

+----+----+------+
|P_id|N_id|weight|
+----+----+------+
|  24|   4|    56|
|  28|   8|    15|
|  26|   6|    16|
|  20|   0|    11|
|  22|   2|     9|
|  25|   5|    45|
|  21|   1|     8|
|  29|   9|    28|
|  23|   3|    12|
+----+----+------+



In [100]:
ss = SparkSession(sc)


In [101]:
df1 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('hdfs://localhost:9000/Final_Project/sample/Sample1_cleaned.csv')
df1.show(5)

+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
| id|     DisplayName|          Location|             AboutMe|Age|       phone|     City|State|Country|
+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
|  0|       Community|on the server farm|Hi, I'm not reall...| 27|655-452-4816|     null| null|   null|
|  1|    Geoff Dalgas|     Corvallis, OR|Dev #2 who helped...| 24|460-380-5210|Corvallis|   OR|   null|
|  2|             Jin|       Raleigh, NC|I design stuff fo...| 35|728-871-7285|  Raleigh|   NC|   null|
|  3|Stefan Lasiewski|      Berkeley, CA|Stefan Lasiewski....| 25|728-363-9979| Berkeley|   CA|   null|
|  4|      txwikinger|            Canada|Ubuntu member, Ku...| 35|343-135-3102|   Canada| null|   null|
+---+----------------+------------------+--------------------+---+------------+---------+-----+-------+
only showing top 5 rows



In [102]:
df2 = ss.read.option("header",True)\
        .option("inferSchema",True)\
        .csv('hdfs://localhost:9000/Final_Project/sample/Sample2_cleaned.csv')
df2.show(5)

+---+----------------+--------------------+--------------------+---+------------+--------------+------------+-------+
| id|     DisplayName|            Location|             AboutMe|Age|       phone|          City|       State|Country|
+---+----------------+--------------------+--------------------+---+------------+--------------+------------+-------+
|  0|           tante|  Oldenburg, Germany|Just some random ...| 23|507-293-0738|     Oldenburg|     Germany|   null|
|  1|Alan Featherston|Wellington, New Z...|       Data Engineer| 29|991-246-5372|    Wellington| New Zealand|   null|
|  2|         jamespo|      United Kingdom|Just another Linu...| 38|665-081-3271|United Kingdom|        null|   null|
|  3|       fwaechter|          Vienna, VA|I'm feeling rough...| 28|923-759-1017|        Vienna|          VA|   null|
|  4|     Marco Ceppi|          Vienna, VA|I'm a Moderator -...| 32|375-304-3599|        Vienna|          VA|   null|
+---+----------------+--------------------+-------------

In [103]:
df2 = df2.withColumnRenamed("id" , "id_2")\
              .withColumnRenamed("DisplayName","DisplayName_2") \
              .withColumnRenamed("Location" , "Location_2")\
              .withColumnRenamed("AboutMe" , "AboutMe_2")\
              .withColumnRenamed("Age" , "Age_2")\
              .withColumnRenamed("phone" , "phone_2")\
              .withColumnRenamed("City" , "City_2")\
              .withColumnRenamed("State" , "State_2")\
              .withColumnRenamed("Country" , "Country_2")
df2.show(5)

+----+----------------+--------------------+--------------------+-----+------------+--------------+------------+---------+
|id_2|   DisplayName_2|          Location_2|           AboutMe_2|Age_2|     phone_2|        City_2|     State_2|Country_2|
+----+----------------+--------------------+--------------------+-----+------------+--------------+------------+---------+
|   0|           tante|  Oldenburg, Germany|Just some random ...|   23|507-293-0738|     Oldenburg|     Germany|     null|
|   1|Alan Featherston|Wellington, New Z...|       Data Engineer|   29|991-246-5372|    Wellington| New Zealand|     null|
|   2|         jamespo|      United Kingdom|Just another Linu...|   38|665-081-3271|United Kingdom|        null|     null|
|   3|       fwaechter|          Vienna, VA|I'm feeling rough...|   28|923-759-1017|        Vienna|          VA|     null|
|   4|     Marco Ceppi|          Vienna, VA|I'm a Moderator -...|   32|375-304-3599|        Vienna|          VA|     null|
+----+----------

In [104]:
#common_df = df2.join(df_weight,df2.id_2 == df_weight.N_id,"inner")

In [105]:
# inner join on two dataframes
common_df = df1.join(df_weight,df1.id == df_weight.P_id,"inner")

In [106]:
common_df

DataFrame[id: int, DisplayName: string, Location: string, AboutMe: string, Age: int, phone: string, City: string, State: string, Country: string, P_id: bigint, N_id: bigint, weight: bigint]

In [107]:
pd_common_df = common_df.toPandas()
pd_common_df

id       DisplayName                         Location  \
0  24       Marco Ceppi                       Vienna, VA   
1  28          jacksonh                       Boston, MA   
2  26       Jim Fiorato  Wheaton, Illinois United States   
3  20             tante                          Germany   
4  22           jamespo                   United Kingdom   
5  25  Benjamin Pollack                              USA   
6  21              Alan          Wellington, New Zealand   
7  29              Dale                              USA   
8  23         fwaechter                       Vienna, VA   

                                             AboutMe  Age         phone  \
0  I'm a Moderator - if you have any issues, ques...   32  375-304-3599   
1  I am a developer on the Mono and Moonlight pro...   35  490-267-2635   
2  I'm very important. I have many leather-bound ...   35  757-448-8875   
3  Just some random coder, for more check out my ...   23  507-293-0738   
4       Just another Linux, Python &amp; Perl hacker   38   123-456-789   
5  I work at Bakpax, where we're focusing on maki...   30  820-002-3042   
6                                      Data Engineer   29  991-246-5372   
7  I am an Electrical Engineer / Software Develop...   18  319-613-6622   
8  I'm feeling rough, I'm feeling raw, I'm in the...   28  923-759-1017   

             City                    State Country  P_id  N_id  weight  
0          Vienna                       VA    None    24     4      56  
1          Boston                       MA    None    28     8      15  
2         Wheaton   Illinois United States    None    26     6      16  
3         Germany                     None    None    20     0      11  
4  United Kingdom                     None    None    22     2       9  
5             USA                     None    None    25     5      45  
6      Wellington              New Zealand    None    21     1       8  
7             USA                     None    None    29     9      28  
8          Vienna                       VA    None    23     3      12

In [108]:
common_df_2 = common_df.join(df2,common_df.N_id == df2.id_2,"inner")
pd_common_df_2 = common_df_2.toPandas()
pd_common_df_2 = pd_common_df_2.drop(["Location","Location_2","P_id","N_id"], axis=1)
pd_common_df_2

id       DisplayName                                            AboutMe  \
0  24       Marco Ceppi  I'm a Moderator - if you have any issues, ques...   
1  28          jacksonh  I am a developer on the Mono and Moonlight pro...   
2  26       Jim Fiorato  I'm very important. I have many leather-bound ...   
3  20             tante  Just some random coder, for more check out my ...   
4  22           jamespo       Just another Linux, Python &amp; Perl hacker   
5  25  Benjamin Pollack  I work at Bakpax, where we're focusing on maki...   
6  21              Alan                                      Data Engineer   
7  29              Dale  I am an Electrical Engineer / Software Develop...   
8  23         fwaechter  I'm feeling rough, I'm feeling raw, I'm in the...   

   Age         phone            City                    State Country  weight  \
0   32  375-304-3599          Vienna                       VA    None      56   
1   35  490-267-2635          Boston                       MA    None      15   
2   35  757-448-8875         Wheaton   Illinois United States    None      16   
3   23  507-293-0738         Germany                     None    None      11   
4   38   123-456-789  United Kingdom                     None    None       9   
5   30  820-002-3042             USA                     None    None      45   
6   29  991-246-5372      Wellington              New Zealand    None       8   
7   18  319-613-6622             USA                     None    None      28   
8   28  923-759-1017          Vienna                       VA    None      12   

   id_2     DisplayName_2                                          AboutMe_2  \
0     4       Marco Ceppi  I'm a Moderator - if you have any issues, ques...   
1     8          jacksonh  I am a developer on the Mono and Moonlight pro...   
2     6       Jim Fiorato  I'm very important. I have many leather-bound ...   
3     0             tante  Just some random coder, for more check out my ...   
4     2           jamespo       Just another Linux, Python &amp; Perl hacker   
5     5  Benjamin Pollack  I work at Bakpax, where we're focusing on maki...   
6     1  Alan Featherston                                      Data Engineer   
7     9        Dale Ragan  I am an Electrical Engineer / Software Develop...   
8     3         fwaechter  I'm feeling rough, I'm feeling raw, I'm in the...   

   Age_2       phone_2          City_2                  State_2  \
0     32  375-304-3599          Vienna                       VA   
1     35  490-267-2635          Boston                       MA   
2     35  757-448-8875         Wheaton   Illinois United States   
3     23  507-293-0738       Oldenburg                  Germany   
4     38  665-081-3271  United Kingdom                     None   
5     30  820-002-3042         Raleigh                       NC   
6     29  991-246-5372      Wellington              New Zealand   
7     18  319-613-6622   United States                     None   
8     28  923-759-1017          Vienna                       VA   

        Country_2  
0            None  
1            None  
2            None  
3            None  
4            None  
5   United States  
6            None  
7            None  
8            None

In [109]:
pd_common_df_2

id       DisplayName                                            AboutMe  \
0  24       Marco Ceppi  I'm a Moderator - if you have any issues, ques...   
1  28          jacksonh  I am a developer on the Mono and Moonlight pro...   
2  26       Jim Fiorato  I'm very important. I have many leather-bound ...   
3  20             tante  Just some random coder, for more check out my ...   
4  22           jamespo       Just another Linux, Python &amp; Perl hacker   
5  25  Benjamin Pollack  I work at Bakpax, where we're focusing on maki...   
6  21              Alan                                      Data Engineer   
7  29              Dale  I am an Electrical Engineer / Software Develop...   
8  23         fwaechter  I'm feeling rough, I'm feeling raw, I'm in the...   

   Age         phone            City                    State Country  weight  \
0   32  375-304-3599          Vienna                       VA    None      56   
1   35  490-267-2635          Boston                       MA    None      15   
2   35  757-448-8875         Wheaton   Illinois United States    None      16   
3   23  507-293-0738         Germany                     None    None      11   
4   38   123-456-789  United Kingdom                     None    None       9   
5   30  820-002-3042             USA                     None    None      45   
6   29  991-246-5372      Wellington              New Zealand    None       8   
7   18  319-613-6622             USA                     None    None      28   
8   28  923-759-1017          Vienna                       VA    None      12   

   id_2     DisplayName_2                                          AboutMe_2  \
0     4       Marco Ceppi  I'm a Moderator - if you have any issues, ques...   
1     8          jacksonh  I am a developer on the Mono and Moonlight pro...   
2     6       Jim Fiorato  I'm very important. I have many leather-bound ...   
3     0             tante  Just some random coder, for more check out my ...   
4     2           jamespo       Just another Linux, Python &amp; Perl hacker   
5     5  Benjamin Pollack  I work at Bakpax, where we're focusing on maki...   
6     1  Alan Featherston                                      Data Engineer   
7     9        Dale Ragan  I am an Electrical Engineer / Software Develop...   
8     3         fwaechter  I'm feeling rough, I'm feeling raw, I'm in the...   

   Age_2       phone_2          City_2                  State_2  \
0     32  375-304-3599          Vienna                       VA   
1     35  490-267-2635          Boston                       MA   
2     35  757-448-8875         Wheaton   Illinois United States   
3     23  507-293-0738       Oldenburg                  Germany   
4     38  665-081-3271  United Kingdom                     None   
5     30  820-002-3042         Raleigh                       NC   
6     29  991-246-5372      Wellington              New Zealand   
7     18  319-613-6622   United States                     None   
8     28  923-759-1017          Vienna                       VA   

        Country_2  
0            None  
1            None  
2            None  
3            None  
4            None  
5   United States  
6            None  
7            None  
8            None

In [110]:
pd_common_df_2.to_csv('/home/aditi/BigData/AAI_Project/dataset/Output.csv', header=True, index=False)

In [111]:
baseDir = 'hdfs://localhost:9000/Final_Project/sample/Output.csv'
common_df_2.write.option("header", True).mode('overwrite').csv(baseDir)

In [97]:
tokens_per_profile = profiles.map(lambda profile: profile.attributes if((profile.profile_id in df["P_id"]) and (df["weight"]>8)) else None)